# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder: fetch latest code</h2>
            <span style="color:#f71;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml --prune</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h1 style="color:#900;">Important Note</h1>
            <span style="color:#900;">
            In this lab, I use GPT-4o and Claude-3.5-Sonnet, which are the slightly higher priced models. The costs are still low, but if you'd prefer to keep costs ultra low, please make the suggested switches to the models (3 cells down from here).
            </span>
        </td>
    </tr>
</table>

In [4]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI, AzureOpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [5]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')

azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_openai_api_endpoint = os.getenv('AZURE_OPENAI_API_ENDPOINT')
if azure_openai_api_key:
    print(f"Azure OpenAI API Key exists and begins {azure_openai_api_key[:8]}")
else:
    print("Azure OpenAI API Key not set") 

Azure OpenAI API Key exists and begins 75Ro1MBh


In [6]:
# initialize
# NOTE - option to use ultra-low cost models by uncommenting last 2 lines

# openai = OpenAI()
openai = AzureOpenAI(
    api_key=azure_openai_api_key,
    api_version="2025-01-01-preview",
    azure_endpoint=azure_openai_api_endpoint
)
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

# Want to keep costs ultra-low? Uncomment these lines:
# OPENAI_MODEL = "gpt-4o-mini"
# CLAUDE_MODEL = "claude-3-haiku-20240307"

In [7]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [8]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [9]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [10]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [11]:
def optimize_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        if chunk.choices and hasattr(chunk.choices[0].delta, 'content'):
            fragment = chunk.choices[0].delta.content or ""
            reply += fragment
            print(fragment, end='', flush=True)
    write_output(reply)

In [12]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)

In [13]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [14]:
exec(pi)

Result: 3.141592658589
Execution Time: 20.449065 seconds


In [15]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <cmath>
#include <chrono>

double calculate(long long iterations, int param1, int param2) {
    double result = 1.0; // Use double for precision
    for (long long i = 1; i <= iterations; ++i) {
        double j1 = static_cast<double>(i * param1 - param2);
        result -= (1.0 / j1);
        double j2 = static_cast<double>(i * param1 + param2);
        result += (1.0 / j2);
    }
    return result;
}

int main() {
    const long long ITERATIONS = 100000000;
    const int PARAM1 = 4;
    const int PARAM2 = 1;

    auto start_time = std::chrono::high_resolution_clock::now();

    double result = calculate(ITERATIONS, PARAM1, PARAM2) * 4.0;

    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> elapsed = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << "\n";
    std::cout << "Execution Time: " << elapsed.count() << " seco

In [16]:
exec(pi)

Result: 3.141592658589
Execution Time: 21.256333 seconds


# Compiling C++ and executing

This next cell contains the command to compile a C++ file on my M1 Mac.  
It compiles the file `optimized.cpp` into an executable called `optimized`  
Then it runs the program called `optimized`

In the next lab (day4), a student has contributed a full solution that compiles to efficient code on Mac, PC and Linux!

You can wait for this, or you can google (or ask ChatGPT!) for how to do this on your platform, then replace the lines below.
If you're not comfortable with this step, you can skip it for sure - I'll show you exactly how it performs on my Mac.


OR alternatively: student Sandeep K.G. points out that you can run Python and C++ code online to test it out that way. Thank you Sandeep!  
> Not an exact comparison but you can still get the idea of performance difference.
> For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [17]:
# Compile C++ and run the executable

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

'clang++' is not recognized as an internal or external command,
operable program or batch file.
'.' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
optimize_claude(pi)

AuthenticationError: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

In [22]:
# Repeat for Claude - again, use the right approach for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

'clang++' is not recognized as an internal or external command,
operable program or batch file.
'.' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [24]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 67.429557 seconds


In [25]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <cstdint>     // For uint32_t
#include <algorithm>   // For std::max
#include <chrono>      // For timing

// Linear Congruential Generator (LCG) function
class LCG {
public:
    LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 4294967296) 
        : value(seed), A(a), C(c), M(m) {}

    uint32_t next() {
        value = (A * value + C) % M;
        return value;
    }

private:
    uint32_t value;
    const uint32_t A, C, M;
};

// Function to calculate maximum subarray sum
int64_t max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> random_numbers(n);

    // Generate random numbers
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg.next() % (max_val - min_val + 1) + min_val;
    }

    // Kadane's algorithm to calculate maximum subarray sum
    int64_t max_sum = INT64_MIN, current_sum = 0;
    for (int i = 0; i < n; ++i) {
        c

In [26]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

'clang++' is not recognized as an internal or external command,
operable program or batch file.
'.' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
optimize_claude(python_hard)

AuthenticationError: Error code: 401 - {'type': 'error', 'error': {'type': 'authentication_error', 'message': 'invalid x-api-key'}}

In [ ]:
# Replace this with the right C++ compile + execute command for your platform

!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

In [ ]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        if chunk.choices and hasattr(chunk.choices[0].delta, 'content'):
            fragment = chunk.choices[0].delta.content or ""
            reply += fragment
            yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [ ]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far        

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

In [ ]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [ ]:
# You'll need to change the code in the try block to compile the C++ code for your platform
# I pasted this into Claude's chat UI with a request for it to give me a version for an Intel PC,
# and it responded with something that looks perfect - you can try a similar approach for your platform.

# M1 Mac version to compile and execute optimized C++ code:

def execute_cpp(code):
        write_output(code)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [ ]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [ ]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)